# Adding Plane Classification to a Voxel51 Dataset
This notebook will pass the samples in a Voxel51 Dataset through a trained model to classify whether there is a plane in the image or not.
This prediction will be added to the sample as a Label and a Tag will be generated.

In [1]:
dataset_name="test-dataset"  # Name of the Voxel51 Dataset to use
model_path = '/tf/dataset/plane-classifier'  # Path to the exported TF Classification Model
img_width, img_height = 299, 299  # The image dimensions that the model expects

In [5]:
import fiftyone as fo
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

dataset = fo.load_dataset(dataset_name)

### Load the model

In [3]:
# load the trained model
model = load_model(model_path)
model.compile()


### Examine the Dataset

In [6]:
print(dataset)

Name:           test-dataset
Media type:     image
Num samples:    42837
Persistent:     False
Info:           {}
Tags:           []
Sample fields:
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)


### Classify the samples

In [7]:
view = dataset.take(10)
for sample in view:
    image_size = (img_width, img_height)
    img = keras.preprocessing.image.load_img(sample.filepath, target_size=image_size)

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = model.predict(img_array)
    score = predictions[0][0]
    #print(score)
    if score>0:
        label="plane"
        sample.tags.append("plane")
    else:
        label="no plane"
        sample.tags.append("noPlane")
    sample["plane"] = fo.Classification(confidence=score,label=label )
    sample.save()


### Examine the results

In [8]:
session = fo.launch_app(dataset, auto=False)

Connected to fiftyone on local port 5151
If you are not connecting to a remote session, you may need
to start a new session and specify a port.

Session launched. Run `session.show()` to open the App in a cell output.


In [9]:
view = dataset.exists("plane")
session.view = view